# Imports

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
#config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

## Env setup

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

## Object detection imports
Here are the imports from the object detection module.

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_CKPT` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.compat.v1.GraphDef()
    with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [6]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [7]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [8]:
import cv2
inputVideo = ''
cap=cv2.VideoCapture(0) # 0 stands for very first webcam attach
#filename="output.avi"
#codec=cv2.VideoWriter_fourcc('m','p','4','v')#fourcc stands for four character code
#framerate=6
#resolution=(640,480)
    
#VideoFileOutput=cv2.VideoWriter(filename,codec,framerate, resolution)

with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph) as sess:
    ret=True
    
    while (ret):
        ret, image_np=cap.read() 
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
          # Actual detection.
        (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
        
        
        for index, value in enumerate(classes[0]):
            ymin = boxes[0][index][0]
            xmin = boxes[0][index][1]
            ymax = boxes[0][index][2]
            xmax = boxes[0][index][3]
            class_name = (category_index.get(value)).get('name')
            widthvalue = int((xmax - xmin) / 2)  # width 길이
            heightvalue = int((ymax - ymin) / 2)  # height 길이
            if (class_name == "person" and scores[0, index] > 0.80) :
                print((xmin + xmax) / 2, (ymin + ymax) / 2)
                print("person"  + str(index) + "\n")
        #VideoFileOutput.write(image_np)
        cv2.imshow('live_detection', image_np)
        if cv2.waitKey(25) & 0xFF==ord('q'):
            break
            cv2.destroyAllWindows()
            cap.release()

UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1 (defined at <ipython-input-5-3d40304b1ed0>:7) ]]
	 [[Postprocessor/BatchMultiClassNonMaxSuppression/mul/_53]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1 (defined at <ipython-input-5-3d40304b1ed0>:7) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'FeatureExtractor/MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1':
  File "C:\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Anaconda3\lib\runpy.py", line 

In [ ]:
import cv2
car_index = [[[], []] for i in range(10)]

inputVideo = 'C:\\Users\\corallines\\Desktop\\VIDEO\\a3.mp4'
cap=cv2.VideoCapture(inputVideo) # 0 stands for very first webcam attach
#filename="output.avi"
#codec=cv2.VideoWriter_fourcc('m','p','4','v')#fourcc stands for four character code
#framerate=6
#resolution=(640,480)
    
#VideoFileOutput=cv2.VideoWriter(filename,codec,framerate, resolution)

with detection_graph.as_default():
  with tf.compat.v1.Session(graph=detection_graph) as sess:
    ret=True
    
    while (ret):
        ret, image_np=cap.read() 
        # Definite input and output Tensors for detection_graph
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        # Each box represents a part of the image where a particular object was detected.
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        # Each score represent how level of confidence for each of the objects.
        # Score is shown on the result image, together with the class label.
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
       
          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
        image_np_expanded = np.expand_dims(image_np, axis=0)
          # Actual detection.
        (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
        
        for index, value in enumerate(classes[0]):
            ymin = boxes[0][index][0]
            xmin = boxes[0][index][1]
            ymax = boxes[0][index][2]
            xmax = boxes[0][index][3]
            class_name = (category_index.get(value)).get('name')
            widthvalue = int((xmax - xmin) / 2)  # width 길이
            heightvalue = int((ymax - ymin) / 2)  # height 길이
            if (class_name == "car" and scores[0, index] > 0.80) :
                x_center = (xmin + xmax) / 2
                y_center = (ymin + ymax) / 2
                                
                car_index[index][0].append(x_center)
                car_index[index][1].append(y_center)
                            
        #VideoFileOutput.write(image_np)
        cv2.imshow('live_detection', image_np)
        if cv2.waitKey(25) & 0xFF==ord('q'):
            break
            cv2.destroyAllWindows()
            cap.release()

In [ ]:
def get_shortest_index(ar, x_c, y_c):
    
    min_distance = 100
    min_index = -1
    for i in range(0, 10):
        if (len(ar[i]) != 0):            
            test_distance = (ar[i][0][-1] - x_c) ** 2 + (ar[i][1][-1] - y_c) ** 2
            if (test_distance < min_distance):
                min_distance = test_distance
                min_index = i
        

In [15]:
import matplotlib.pyplot as plt
plt.title("Car Center Route")

#plt.xlim([0,1]) 
#plt.ylim([0,1]) 

plt.gca().invert_yaxis()

plt.plot(car_index[0][0], car_index[0][1], 'red')
plt.plot(car_index[1][0], car_index[1][1], 'blue')
plt.plot(car_index[2][0], car_index[2][1], 'green')
plt.plot(car_index[3][0], car_index[3][1], 'black')
plt.show()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  del sys.path[0]


In [11]:
len(car_index[0][0])

133

In [13]:
print(car_index[0][0])
print(car_index[0][0][-1])


[0.1823110580444336, 0.13694322109222412, 0.18480542302131653, 0.1427447497844696, 0.18065699934959412, 0.1830846518278122, 0.17578332126140594, 0.14233314990997314, 0.14219266176223755, 0.141607403755188, 0.1388419270515442, 0.17692972719669342, 0.1754152625799179, 0.1751893013715744, 0.17703275382518768, 0.17739351093769073, 0.17762041091918945, 0.1828448325395584, 0.1814310997724533, 0.1823435127735138, 0.18063759803771973, 0.14488191902637482, 0.14774349331855774, 0.1455456018447876, 0.14628633856773376, 0.18034788966178894, 0.1425664871931076, 0.18193311989307404, 0.1834823489189148, 0.18386808037757874, 0.18350444734096527, 0.17850211262702942, 0.1788490116596222, 0.18110871315002441, 0.18421994149684906, 0.18518827855587006, 0.18500830233097076, 0.1832357496023178, 0.18389427661895752, 0.18477898836135864, 0.18525201082229614, 0.1870577484369278, 0.18687985837459564, 0.18454018235206604, 0.1829342246055603, 0.1833690106868744, 0.18460290133953094, 0.1832607388496399, 0.182438179

In [14]:

plt.show()


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  """Entry point for launching an IPython kernel.
